In [1]:
# 案
# misconceptionからランダムに10件データをとってくる
# misconceptionを10件見せるから、それに合うようなhoge, fuga, piyoを生成してください
# 例を5つ(ランダムに選択)与えるため、参考にしてください。

# 結果
# ランダムに10件与えらデータから、良い感じに選んでくることができないため、中止

In [2]:
# まずは、どのようなpromptを投げたら、良い感じの誤解に基づいた問題を生成してくれるかを調べる。
import polars as pl

In [3]:
train = pl.read_csv("../../data/train.csv")
misconception_mapping = pl.read_csv("../../data/misconception_mapping.csv")

In [4]:
misconception_mapping_dict = dict(
    zip(
        misconception_mapping["MisconceptionId"],
        misconception_mapping["MisconceptionName"],
    )
)

In [5]:
# Misconception[A-D]Nameを追加する
train = train.with_columns(
    [
        pl.col(f"Misconception{word}Id")
        .cast(pl.Int64)
        .replace_strict(misconception_mapping_dict)
        .alias(f"Misconception{word}Name")
        for word in "ABCD"
    ]
)

In [7]:
# 行単位では行えない、元々のデータだと誤解は４つあるが、誤解からデータを生み出す場合には、
def make_prompt(df: pl.DataFrame, idx: int):
    row = df[idx]

    construct_name = row["ConstructName"][0]
    subject_name = row["SubjectName"][0]
    question_text = row["QuestionText"][0]
    correct_answer = row["CorrectAnswer"][0]

    answer_a_text = row["AnswerAText"][0]
    answer_b_text = row["AnswerBText"][0]
    answer_c_text = row["AnswerCText"][0]
    answer_d_text = row["AnswerDText"][0]

    misconception_a_name = row["MisconceptionAName"][0]
    misconception_b_name = row["MisconceptionBName"][0]
    misconception_c_name = row["MisconceptionCName"][0]
    misconception_d_name = row["MisconceptionDName"][0]

    prompt = f"""
    I will now provide 10 misconception candidates. 
    Please select 1-3 out of 10 misconceptions and create problems that could lead to misconceptions.
    Please create ConstructName, SubjectName, QuestionText, CorrectAnswer, AnswerAText, AnswerBText, AnswerCText, and AnswerDText. 
    I will provide 1 examples, so please use them as a reference.

    # misconception candidates
    'When measuring a reflex angle, gives the acute or obtuse angle that sums to 360 instead',
    'Believes gradient = change in y',
    'Believes duration can be read from a timetable, rather than calculating from start and end times',
    'Thinks that the side view does not include the furthest rows back',
    'Does not subtract from the hours, when having to borrow for a time calculation',
    'Believes all numbers written in standard form must be smaller than any numbers written in ordinary form',
    'Thinks that the scale on axes has to match with the number of squares',
    'Assumes there are only 2 terms in the nth term formula for a quadratic sequence',
    'Ignores unlabelled angles when applying angle sum rules',
    'When solving simultaneous equations graphically, they think the solutions are the x or y intercepts'
    
    # example
    Question: {question_text}
    Correct Answer: {correct_answer}
    Construct Name: {construct_name}
    Subject Name: {subject_name}

    Answer A Text: {answer_a_text}
    Answer B Text: {answer_b_text}
    Answer C Text: {answer_c_text}
    Answer D Text: {answer_d_text} 

    Misconception A Name: {misconception_a_name}
    Misconception B Name: {misconception_b_name}
    Misconception C Name: {misconception_c_name}
    Misconception D Name: {misconception_d_name}
    """
    print(prompt)


make_prompt(train, 1)



    I will now provide 10 misconception candidates. 
    Please select 1-3 out of 10 misconceptions and create problems that could lead to misconceptions.
    Please create ConstructName, SubjectName, QuestionText, CorrectAnswer, AnswerAText, AnswerBText, AnswerCText, and AnswerDText. 
    I will provide 1 examples, so please use them as a reference.

    # misconception candidates
    'When measuring a reflex angle, gives the acute or obtuse angle that sums to 360 instead',
    'Believes gradient = change in y',
    'Believes duration can be read from a timetable, rather than calculating from start and end times',
    'Thinks that the side view does not include the furthest rows back',
    'Does not subtract from the hours, when having to borrow for a time calculation',
    'Believes all numbers written in standard form must be smaller than any numbers written in ordinary form',
    'Thinks that the scale on axes has to match with the number of squares',
    'Assumes there are only 2

In [ ]:
train  # [0]["ConstructName"][0]

In [ ]:
# 学習データに対し、それぞれのmisconception_idをnameに変換した列を追加する
# hoge, fuga, piyo、answer